In [7]:
import gym
import random

In [8]:
env = gym.make("SpaceInvaders-v0", render_mode='human')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

/home/prathamesh/.local/lib/python3.10/site-packages/gym/envs/registration.py:564: UserWarning: WARN: The environment SpaceInvaders-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]
/home/prathamesh/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


In [9]:
actions

6

In [35]:
episodes = 10
for i in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
#         env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score +=reward
    print(f"episode :{i}, score : {score}")
# env.close()

episode :1, score : 185.0
episode :2, score : 395.0
episode :3, score : 225.0
episode :4, score : 115.0
episode :5, score : 55.0
episode :6, score : 65.0
episode :7, score : 215.0
episode :8, score : 360.0
episode :9, score : 55.0
episode :10, score : 260.0


In [10]:
import numpy as np
import tensorflow
# from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [11]:
def build_model(height, width, channels, actions):
    model = tensorflow.keras.Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape = (3, height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [12]:
model = build_model(height, width, channels, actions)

2022-07-04 12:02:42.743808: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-04 12:02:42.791828: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-04 12:02:42.792026: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-04 12:02:42.792600: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten (Flatten)           (None, 67584)             0         
                                                                 
 dense (Dense)               (None, 512)               34603520  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 6)                 1

In [14]:
from rl.agents import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory

In [19]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [20]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

RecursionError: maximum recursion depth exceeded while getting the str of an object

In [30]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...


/home/prathamesh/.local/lib/python3.10/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2022-07-03 21:43:57.868232: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


  497/10000: episode: 1, duration: 45.464s, episode steps: 497, steps per second:  11, episode reward: 45.000, mean reward:  0.091 [ 0.000, 15.000], mean action: 2.419 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
  888/10000: episode: 2, duration: 33.987s, episode steps: 391, steps per second:  12, episode reward: 35.000, mean reward:  0.090 [ 0.000, 15.000], mean action: 2.540 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


/home/prathamesh/.local/lib/python3.10/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 1285/10000: episode: 3, duration: 219.043s, episode steps: 397, steps per second:   2, episode reward: 60.000, mean reward:  0.151 [ 0.000, 15.000], mean action: 2.695 [0.000, 5.000],  loss: 4.267679, mean_q: 3.733066, mean_eps: 0.897175
 1672/10000: episode: 4, duration: 279.136s, episode steps: 387, steps per second:   1, episode reward: 65.000, mean reward:  0.168 [ 0.000, 20.000], mean action: 2.478 [0.000, 5.000],  loss: 0.514595, mean_q: 2.645528, mean_eps: 0.866980
 2053/10000: episode: 5, duration: 267.903s, episode steps: 381, steps per second:   1, episode reward: 80.000, mean reward:  0.210 [ 0.000, 20.000], mean action: 2.546 [0.000, 5.000],  loss: 0.538407, mean_q: 2.546086, mean_eps: 0.832420
 2751/10000: episode: 6, duration: 483.512s, episode steps: 698, steps per second:   1, episode reward: 140.000, mean reward:  0.201 [ 0.000, 25.000], mean action: 2.623 [0.000, 5.000],  loss: 0.627084, mean_q: 2.822905, mean_eps: 0.783865
 3143/10000: episode: 7, duration: 272.519s

In [33]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 85.000, steps: 395
Episode 2: reward: 20.000, steps: 380
Episode 3: reward: 185.000, steps: 534
Episode 4: reward: 110.000, steps: 628
Episode 5: reward: 235.000, steps: 994
Episode 6: reward: 210.000, steps: 861
Episode 7: reward: 305.000, steps: 931
Episode 8: reward: 210.000, steps: 826
Episode 9: reward: 445.000, steps: 928
Episode 10: reward: 245.000, steps: 1764
205.0


In [34]:
dqn.save_weights('SavedWeights/10k-SpaceInvaders/dqn_weights.h5f')